In [5]:
import numpy as np
import pandas as pd

In [6]:
movies = pd.read_csv('movies.csv')
credits = pd.read_csv('credits.csv')

In [7]:
movies = pd.merge(movies, credits)

In [8]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [9]:
movies.dropna(inplace = True)

In [11]:
import ast
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [12]:
movies['genres'] = movies['genres'].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [13]:
movies['keywords']= movies['keywords'].apply(convert)

In [14]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [15]:
movies['cast']= movies['cast'].apply(convert3)

In [16]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [17]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [18]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [19]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [20]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())


In [21]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


In [22]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])


In [23]:
#new.head()
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [24]:
new['tags'] = new['tags'].apply(lambda x:x.lower())

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    

In [26]:
vector = cv.fit_transform(new['tags']).toarray()

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
similarity = cosine_similarity(vector)
similarity
new[new['title'] == 'The Lego Movie'].index[0]

744

In [29]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:7]:
        print(new.iloc[i[0]].title)

In [30]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Ender's Game
Aliens vs Predator: Requiem
Independence Day
Krull


In [31]:
import nltk

In [32]:
from nltk.stem.porter import PorterStemmer

In [33]:
ps =  PorterStemmer()

In [34]:
def stem(text):
    y =[]
    
    for i in text.split():
        y.append(ps.stem(i))
        
        
    return " ".join(y)
        
    

In [35]:
ps.stem('heroine')

'heroin'

In [36]:
new['tags'] = new['tags'].apply(stem)

In [37]:
!pip install fuzzywuzzy

In [43]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Assuming 'new' is your DataFrame with columns 'movie_id', 'title', and 'tags'
# Assuming 'similarity' is your similarity matrix

def recommend(movie):
    # Search for movie titles with typo tolerance
    matches = process.extract(movie, new['title'], scorer=fuzz.token_set_ratio)
    relevant_matches = [match for match in matches if match[1] >= 80]
    
    if relevant_matches:
        # Get the closest matching title
        closest_match = relevant_matches[0][0]
        index = new[new['title'] == closest_match].index[0]
        
        distances = sorted(enumerate(similarity[index]), reverse=True, key=lambda x: x[1])
        recommended_movies = [new.iloc[i[0]].title for i in distances[1:6]]
        
        return recommended_movies
    else:
        return "Movie not found"



In [44]:
recommend('a beautifulzs kind')

['Twin Falls Idaho', 'Ramanujan', 'Thirteen', 'The Muse', 'Beyond Borders']

In [45]:
import pickle 

In [46]:
pickle.dump(new,open('movie2_dict.pkl','wb'))

In [47]:
pickle.dump(similarity,open('similarity.pkl','wb'))